In [2]:
import json
import ast
import pip
def install(package):
   pip.main(['install', package])

install('BeautifulSoup4')
from bs4 import BeautifulSoup
import re
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from nltk.corpus import stopwords 

In [3]:
import numpy as np

In [7]:
def data_clean(file_name, num_reviews):
    count = 0
    output = []
    output_text = [] 
    stops = set(stopwords.words("english"))
    with open(file_name) as f:
        while count < num_reviews:
            line = f.next()
            line = ast.literal_eval(line)
            line = json.dumps(line)
            line = json.loads(line)
            text = line["text"].encode('utf-8').strip().replace("\n","")
            text = re.sub(r"http\S+", "", line['text'])
            text = re.sub(r"@\S+", "", text)
            text = re.sub(r"#\S+", "", text)
            text = text.split()
            text = [w for w in text if not text in stopwords.words("english")]
            text = ' '.join(text)
            output.append((line["id"], text))
            output_text.append(text)
            count += 1
    return output, output_text

In [8]:
output_paris, output_text_paris = data_clean("../data/paris_shooting.txt", 1000)

In [9]:
output_airasia, output_text_airasia = data_clean("../data/airasia_crash.txt", 1000)

In [10]:
output_irene, output_text_irene = data_clean("../data/irene_hurricane.txt", 1000)

In [11]:
output_irene[0]

(107146911668109313, u'hurricane irene can suck it so hard.')

In [12]:
output_japan, output_text_japan = data_clean("../data/japan_tsunami.txt", 1000)

In [13]:
output_japan[0]

(1536472088,
 u'RT Massive earthquake hits Japan. Here& how you can help victims of the earthquake and tsunami in Japan. ...')

In [14]:
output_london, output_text_london = data_clean("../data/london_riot.txt", 1000)

In [15]:
output_london[0]

(101044899578785793,
 u'Top story: get your tank refill now BBC News - Live: UK riots see more')

In [16]:
def loop(output_text, list_input):
    for i in range(len(list_input)):
        output_text.append(list_input[i])

In [17]:
output_text = []
loop(output_text, output_text_paris)
loop(output_text, output_text_airasia)
loop(output_text, output_text_irene)
loop(output_text, output_text_japan)
loop(output_text, output_text_london)


In [18]:
len(output_text)

5000

In [19]:
f = open('training_set.txt','w',)
for i in output_text:
    f.write(i.encode('utf-8'))
    f.write('\n')
f.close()

In [20]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None,max_features = 500)
train_data_features = vectorizer.fit_transform(output_text)
train_data_features = train_data_features.toarray()

In [21]:
train_data_features.shape

(5000, 500)

In [22]:
tfidf = TfidfTransformer()
train_data_features_tfidf = tfidf.fit_transform(train_data_features)
train_data_features_tfidf = train_data_features_tfidf.toarray()

In [23]:
pca = PCA(n_components=10)
train_data_features_tfidf=pca.fit_transform(train_data_features_tfidf)

In [24]:
train_data_features_tfidf.shape

(5000, 10)

In [ ]:
kmeans = KMeans(n_clusters=5,n_jobs=-1).fit(train_data_features_tfidf)

In [ ]:
type(train_data_features_tfidf[0])

In [ ]:
kmeans.cluster_centers_

In [98]:
distance_to_centroid_1 = []
for i in range(len(train_data_features_tfidf)):
    distance_to_centroid_1.append(np.linalg.norm(train_data_features_tfidf[i]-kmeans.cluster_centers_[0]))

In [99]:
distance_to_centroid_2 = []
for i in range(len(train_data_features_tfidf)):
    distance_to_centroid_2.append(np.linalg.norm(train_data_features_tfidf[i]-kmeans.cluster_centers_[1]))

In [100]:
distance_to_centroid_3 = []
for i in range(len(train_data_features_tfidf)):
    distance_to_centroid_3.append(np.linalg.norm(train_data_features_tfidf[i]-kmeans.cluster_centers_[2]))

In [101]:
distance_to_centroid_4 = []
for i in range(len(train_data_features_tfidf)):
    distance_to_centroid_4.append(np.linalg.norm(train_data_features_tfidf[i]-kmeans.cluster_centers_[3]))

In [102]:
distance_to_centroid_5 = []
for i in range(len(train_data_features_tfidf)):
    distance_to_centroid_5.append(np.linalg.norm(train_data_features_tfidf[i]-kmeans.cluster_centers_[4]))

In [103]:
distance_to_centroid_1.index(min(distance_to_centroid_1))

3212

In [108]:
output_text[3212]

u'Whoa, Just Seen Some Footy From The Japan Shit. Damn, Looks Like Some Movie Shit. i Can& Even Think Of A joke Right Now.'

In [104]:
distance_to_centroid_2.index(min(distance_to_centroid_2))

1259

In [109]:
output_text[1259]

u"Tangeraanngg i'm comiinnggg!! \U0001f601\U0001f633 btw thanks\u2026 (w/ Akbar at Juanda International Airport Surabaya Terminal 2) \u2014"

In [105]:
distance_to_centroid_3.index(min(distance_to_centroid_3))

1101

In [110]:
output_text[1101]

u'RT "AirAsia Flight 8501 has lost contact with air traffic control in Jakarta, Indonesia, the airline says." via'

In [106]:
distance_to_centroid_4.index(min(distance_to_centroid_4))

641

In [111]:
output_text[641]

u'France hunts for gunmen who killed 12 people at Paris magazine'

In [107]:
distance_to_centroid_5.index(min(distance_to_centroid_5))

1805

In [112]:
output_text[1805]

u'RT Air Asia flight QZ 8501 to Singapore loses contact, 155 passengers on board'

In [113]:
predicted = kmeans.predict(train_data_features_tfidf)

In [115]:
summary_paris = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(1000):
    if predicted[i] == 0:
        count_0 += 1
    elif predicted[i] == 1:
        count_1 += 1
    elif predicted[i] == 2:
        count_2 += 1
    elif predicted[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_paris[0] = count_0
summary_paris[1] = count_1
summary_paris[2] = count_2
summary_paris[3] = count_3
summary_paris[4] = count_4
print summary_paris

{0: 596, 1: 0, 2: 0, 3: 404, 4: 0}


In [116]:
summary_AirAsia = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(1000,2000):
    if predicted[i] == 0:
        count_0 += 1
    elif predicted[i] == 1:
        count_1 += 1
    elif predicted[i] == 2:
        count_2 += 1
    elif predicted[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_AirAsia[0] = count_0
summary_AirAsia[1] = count_1
summary_AirAsia[2] = count_2
summary_AirAsia[3] = count_3
summary_AirAsia[4] = count_4
print summary_AirAsia

{0: 307, 1: 103, 2: 236, 3: 0, 4: 354}


In [117]:
summary_Japan = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(2000,3000):
    if predicted[i] == 0:
        count_0 += 1
    elif predicted[i] == 1:
        count_1 += 1
    elif predicted[i] == 2:
        count_2 += 1
    elif predicted[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_Japan[0] = count_0
summary_Japan[1] = count_1
summary_Japan[2] = count_2
summary_Japan[3] = count_3
summary_Japan[4] = count_4
print summary_Japan

{0: 1000, 1: 0, 2: 0, 3: 0, 4: 0}


In [118]:
summary_London = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(3000,4000):
    if predicted[i] == 0:
        count_0 += 1
    elif predicted[i] == 1:
        count_1 += 1
    elif predicted[i] == 2:
        count_2 += 1
    elif predicted[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_London[0] = count_0
summary_London[1] = count_1
summary_London[2] = count_2
summary_London[3] = count_3
summary_London[4] = count_4
print summary_London

{0: 1000, 1: 0, 2: 0, 3: 0, 4: 0}


In [119]:
summary_Irene = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(3000,4000):
    if predicted[i] == 0:
        count_0 += 1
    elif predicted[i] == 1:
        count_1 += 1
    elif predicted[i] == 2:
        count_2 += 1
    elif predicted[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_Irene[0] = count_0
summary_Irene[1] = count_1
summary_Irene[2] = count_2
summary_Irene[3] = count_3
summary_Irene[4] = count_4
print summary_Irene

{0: 1000, 1: 0, 2: 0, 3: 0, 4: 0}


In [84]:
kmeans.transform(train_data_features_tfidf)

array([[ 0.91261295,  1.26007273,  1.24407395,  0.62881109,  1.03383773],
       [ 0.91261295,  1.26007273,  1.24407395,  0.62881109,  1.03383773],
       [ 0.440411  ,  0.97830052,  0.95916945,  0.34366354,  0.65933913],
       ..., 
       [ 0.23236274,  0.90270942,  0.8984772 ,  0.57397626,  0.55234505],
       [ 0.28457291,  0.92099734,  0.89860009,  0.62333369,  0.64951875],
       [ 0.17321711,  0.88147161,  0.88334459,  0.57708883,  0.58967053]])

In [73]:
train_data_features_tfidf[0]

array([-0.12883303,  0.68326897, -0.19917747, -0.17195979,  0.04804421,
        0.31996497, -0.02112298, -0.20635099,  0.09328551,  0.26723798])

In [66]:
len(kmeans)

5000

In [67]:
kmeans[0]

4

In [69]:
len(labels)

0

In [62]:
f = open('labels.txt', 'w')
for i in range(len(labels)):
    f.write(labels[i])
f.close()

In [63]:
for i in range(len(labels)):
    print(labels[i])